## Load a crystal structure from a VASP file format

In [3]:
import os
import numpy as np

### read CONTCAR (Final relaxed structure):

In [4]:
%%time 
#4 sec
from pymatgen.core.structure import Structure
structures=[]
def contcar_reader():
    contcar_file_path = 'CONTCAR'
    # Load the structure from the VASP CONTCAR file
    structure = Structure.from_file(contcar_file_path)
    return structure

for i in range(576):
    vasp_file_path = '/nas/longleaf/home/jarkeith/VASP/GAP_copy/optimize_system_1/output/system_'+str(i)
    os.chdir(vasp_file_path)
    structures.append(contcar_reader())

CPU times: user 3 s, sys: 393 ms, total: 3.39 s
Wall time: 12.7 s


### read OUTCAR (Standard output file):
https://pymatgen.org/pymatgen.io.vasp.outputs.html#class-pymatgeniovaspoutputsoutcarfilename

In [5]:
%%time 
#8 mins
from pymatgen.io.vasp.outputs import Outcar
outcars=[]
errors_occurred=[]
def read_outcar_regular_parameters(i):
    """
    Read the regular parameters from the OUTCAR file.
        
    Returns:
        dict: A dictionary containing the regular parameters.
    """
    try:
        outcar_file_path = 'OUTCAR'
        outcar = Outcar(outcar_file_path)
        regular_parameters = {
            "Total CPU time used (sec)": outcar.run_stats["Total CPU time used (sec)"],
            "System time (sec)": outcar.run_stats["System time (sec)"],
            "Elapsed time (sec)": outcar.run_stats["Elapsed time (sec)"],
            "cores": outcar.run_stats["cores"],
            "total_magmom": outcar.total_mag,
            "final_energy": outcar.final_energy,
            "efermi": outcar.efermi,    }
        return regular_parameters
    except Exception as e:
        print(f"Error occurred for i = {i}: {e}")
        errors_occurred.append(i)
        return None


for i in range(576):
    vasp_file_path = '/nas/longleaf/home/jarkeith/VASP/GAP_copy/optimize_system_1/output/system_'+str(i)
    os.chdir(vasp_file_path)
    outcars.append(read_outcar_regular_parameters(i))

# Now you can access the regular parameters as needed
#for key, value in regular_parameters.items():
#    print(f"{key}: {value}")

print("the number of errors occured: ", len(errors_occurred))
# Create a list of integers from 0 to 575
      
list_of_good_calculations = list(range(576))

# Remove the values in errors_occurred from all_integers using list comprehension
list_of_good_calculations = [x for x in list_of_good_calculations if x not in errors_occurred]



Error occurred for i = 120: 'Total CPU time used (sec)'
Error occurred for i = 122: 'Total CPU time used (sec)'
Error occurred for i = 123: 'Total CPU time used (sec)'
Error occurred for i = 124: 'Total CPU time used (sec)'
Error occurred for i = 125: 'Total CPU time used (sec)'
Error occurred for i = 126: 'Total CPU time used (sec)'
Error occurred for i = 127: 'Total CPU time used (sec)'
Error occurred for i = 128: 'Total CPU time used (sec)'
Error occurred for i = 129: 'Total CPU time used (sec)'
Error occurred for i = 131: 'Total CPU time used (sec)'
Error occurred for i = 132: 'Total CPU time used (sec)'
Error occurred for i = 134: 'Total CPU time used (sec)'
Error occurred for i = 136: 'Total CPU time used (sec)'
Error occurred for i = 137: 'Total CPU time used (sec)'
Error occurred for i = 138: 'Total CPU time used (sec)'
Error occurred for i = 140: 'Total CPU time used (sec)'
Error occurred for i = 324: 'Total CPU time used (sec)'
the number of errors occured:  17
CPU times: use

## Read Vasprun Object (for energies):

### `ionic_steps`:
`ionic_steps` is an attribute of the `Vasprun` object, representing a list of dictionaries. Each dictionary corresponds to the data of a specific ionic step during the ionic relaxation process. The dictionary contains various data related to the ionic step, such as ionic forces, electronic steps, total energies, etc. You can access different quantities of interest for each ionic step using the appropriate keys.

### `ionic_energies`:
`ionic_energies` is a separate list that contains the total energies of each ionic step. This list is derived from the `ionic_steps` attribute by extracting the total energy for each ionic step. In VASP, the total energy for each ionic step is recorded as the sum of the electronic energy, entropy, and other contributions (E_wo_entrp in the vasprun.xml file). The `ionic_energies` list provides an easy way to access and analyze the total energy values without the need to process the `ionic_steps` attribute directly.

---

**Energies for a Specific Ionic Step:**

Each value corresponds to a different type of energy associated with the electronic and ionic configurations of the system. All these energies are typically reported in electron volts (eV) and are used to monitor the convergence and stability of the electronic and ionic configurations during a VASP calculation. They help track the progress of the optimization or relaxation process and provide insight into the energy landscape of the system as it undergoes structural changes.

- `'e_fr_energy'`: Electronic energy of the system at a particular ionic step, including the free energy contribution. It is typically calculated as the total energy of the electronic configuration, including the zero-point energy correction.

- `'e_wo_entrp'`: Total energy of the system at the ionic step, excluding the entropy contribution. It does not account for the vibrational entropy of the system.

- `'e_0_energy'`: Energy of the system at the ionic step, also known as enthalpy. It includes the electronic energy as well as the entropic contribution due to the vibrational modes of the system at 0 Kelvin.

- `'alphaZ'`: Augmentation factor used in the Ewald summation for the electrostatic interactions. It is related to the Coulomb energy.

- `'ewald'`: Ewald sum of the electrostatic interactions between periodic images of the charge distribution.

- `'hartreedc'`: Hartree energy contribution from the density charge.

- `'XCdc'`: Exchange-correlation energy contribution from the density charge.

- `'pawpsdc'`: Partial atomic wave projection of the charge density.

- `'pawaedc'`: Partial atomic wave projection of the augmentation charge.

- `'eentropy'`: Electronic entropy contribution to the free energy.

- `'bandstr'`: Energy contribution from the band structure calculation.

- `'atom'`: Energy contribution from the isolated atoms.

- `'e_fr_energy'`: Electronic free energy of the system at a particular electronic step.

- `'e_wo_entrp'`: Total electronic energy at the electronic step, excluding the electronic entropy contribution.

- `'e_0_energy'`: Energy of the system at the electronic step, including the electronic entropy contribution.


In [6]:
%%time 
# 2 mins
from pymatgen.io.vasp.outputs import Vasprun
import os

def vasprun_reader(i):
    vasprun = Vasprun(filename="vasprun.xml", ionic_step_skip=None, ionic_step_offset=0, parse_dos=True, parse_eigen=True, parse_projected_eigen=False, parse_potcar_file=False, occu_tol=1e-08, separate_spins=False, exception_on_bad_xml=True)

    ionic_step_data = vasprun.ionic_steps[-1]
    electronic_steps = ionic_step_data['electronic_steps']

    e_fr_energy = ionic_step_data['e_fr_energy']
    e_wo_entrop = ionic_step_data['e_wo_entrp']
    e_0_energy = ionic_step_data['e_0_energy']
    forces = ionic_step_data['forces']
    stress = ionic_step_data['stress']
    ewald_value = electronic_steps[0]['ewald']  #also 0 , 12 works.. what one to use?
    hartreedc_value = electronic_steps[0]['hartreedc']
    XCdc_value = electronic_steps[0]['XCdc']
    pawpsdc_value = electronic_steps[0]['pawpsdc']
    pawaedc_value = electronic_steps[0]['pawaedc']
    eentropy_value = electronic_steps[0]['eentropy']
    bandstr_value = electronic_steps[0]['bandstr']
    atom_value = electronic_steps[0]['atom']
    
    data = {
        #'e_fr_energy': [e_fr_energy][0],
        'e_fr_energy': e_fr_energy,
        #'e_wo_entrop': [e_wo_entrop][0],
        #'e_0_energy': [e_0_energy][0],
        #'forces': [forces][0],  # Assuming forces is a list of vectors
        #'stress': [stress][0],  # Assuming stress is a list of values
        ##'coordinates': [coordinates],  # Assuming coordinates is a list of vectors
        #'ewald_value': [ewald_value][0],
        #'hartreedc_value': [hartreedc_value][0],
        #'XCdc_value': [XCdc_value][0],
        #'pawpsdc_value': [pawpsdc_value][0],
        #'pawaedc_value': [pawaedc_value][0],
        #'eentropy_value': [eentropy_value][0],
        #'bandstr_value': [bandstr_value][0],
        #'atom_value': [atom_value][0],
    }

    return data


vaspruns = []
errors_occurred = []

for i in list_of_good_calculations:
    vasp_file_path = '/nas/longleaf/home/jarkeith/VASP/GAP_copy/optimize_system_1/output/system_' + str(i)
    os.chdir(vasp_file_path)
    vaspruns.append(vasprun_reader(i))


# Each element of ionic_steps is a dictionary containing data for each ionic step
# The energy for each step is extracted using "e_wo_entrp" key in the dictionary

# Now you can use ionic_steps and ionic_energies as needed
#for step, energy in zip(range(1, 5), e_0_energy):
#    print(f"Ionic Step: {step}, Energy: {energy:.6f} eV")
#print("...")



CPU times: user 1min 25s, sys: 813 ms, total: 1min 26s
Wall time: 1min 30s


# Define featurizer

- Each value in the 'descriptors' array corresponds to one specific feature computed for the given structure. The exact meaning and interpretation of each feature may vary based on the specific sub-method used and the selected options during the initialization of the `JarvisCFID` instance.

#### [use_cell]
- Cell Descriptors (4 features, based on DensityFeatures and log volume per atom)
- These features describe the lattice parameters and volume of the unit cell of the structure. They are computed using the `DensityFeatures` class and the logarithm of volume per atom.

#### [use_chem]
- Chemical Composition Descriptors (438 features)
- These features describe the chemical composition of the structure. They are based on the elements present in the structure and their fractional composition.

#### [use_chg]
- Core Charge Descriptors (378 features)
- These features represent the core charge descriptors for the elements in the structure. Core charges refer to the charges of the central nuclei in the elements.

#### [use_rdf]
- Radial Distribution Function (RDF) (100 features)
- RDF describes the distribution of atomic pairs with respect to their distances in the structure.

#### [use_adf] 
- Angular Distribution Function (ADF) (179 features x 2, one set of features for each cutoff)
- ADF describes the distribution of angles formed by three atoms in the structure. There are two sets of ADF features, each corresponding to a different cutoff distance.

#### [use_ddf]  
- Dihedral Angle Distribution Function (DDF) (179 features)
- DDF represents the distribution of dihedral angles formed by four atoms in the structure.

#### [use_nn] 
- Nearest Neighbors (100 descriptors)
- These descriptors represent the nearest neighbors of each atom in the structure.


In [7]:
from matminer.featurizers.structure.composite import JarvisCFID
featurizer = JarvisCFID(use_cell=False, use_chem=False, use_chg=False, use_rdf=True, use_adf=True, use_ddf=False, use_nn=False)

## Compute the descriptors for the given structure


In [8]:
%%time 
#13 mins
#You can pass a Structure object to the featurize method to get the chemo-structural CFID descriptors.

#print(descriptors)
descriptors=[]
for i in list_of_good_calculations:
    descriptors.append(featurizer.featurize(structures[i]))

print("the number of descriptors: ", len(descriptors))

the number of descriptors:  559
CPU times: user 12min 23s, sys: 959 ms, total: 12min 24s
Wall time: 12min 25s


## Create the dataframe and save it to a csv file (and remove any features that have all zeros)

In [9]:
import pandas as pd

df=pd.DataFrame(vaspruns)
df_1=pd.DataFrame(descriptors)
# Add the new columns to the existing DataFrame
df = pd.concat([df, df_1], axis=1)

#Remove zeros
columns_with_zeros = df.columns[df.isin([0]).any()]
df = df.drop(columns=columns_with_zeros)
print(columns_with_zeros)

path = r'/nas/longleaf/home/jarkeith/Pymatgen_Examples/matminer'
os.chdir(path)
# Save DataFrame to a CSV file
df.to_csv('output.csv', index=False)  # Set index=False if you don't want to save the row numbers

Index([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
       ...
       448, 449, 450, 451, 452, 453, 454, 455, 456, 457],
      dtype='object', length=226)


## Read in the DataFrame from the .CSV file


In [10]:
# Read in DataFrame from CSV file
import pandas as pd
df = pd.read_csv('output.csv')

data=df

print('Loaded {} entries'.format(len(data)))
data

Loaded 559 entries


,e_fr_energy,10,11,12,13,14,15,17,18,20,...,384,385,386,387,388,389,390,397,398,400
0,-1631.095222,5.7942,1.8904,0.7112,0.7622,21.5403,7.7488,1.9962,0.3248,0.5290,...,6.0,9.0,22.0,12.0,18.0,10.0,3.0,2.0,2.857143,7.0
1,-1631.213548,5.0022,2.1846,0.6724,0.7207,20.3669,7.3266,1.8874,0.3071,0.3126,...,5.0,10.0,23.0,7.0,19.0,11.0,3.0,2.0,2.857143,4.0
2,-1631.249296,5.7232,1.8672,0.7025,0.7529,21.2765,7.6539,1.9717,0.3208,0.3919,...,3.0,12.0,22.0,5.0,23.0,7.0,4.0,2.0,2.857143,6.0
3,-1631.199946,5.6264,1.8356,0.6906,0.7401,20.9165,7.5244,2.0265,0.2365,0.4495,...,4.0,10.0,21.0,9.0,19.0,10.0,5.0,2.0,2.857143,5.0
4,-1631.144332,5.8677,1.9143,0.7202,0.7719,21.8136,7.8471,2.0215,0.3289,0.4688,...,4.0,12.0,22.0,9.0,19.0,9.0,4.0,2.0,2.857143,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
554,-1631.227953,6.2558,1.4604,0.7064,0.7571,21.3957,7.6967,2.0729,0.2419,0.3941,...,6.0,10.0,18.0,16.0,14.0,12.0,4.0,2.0,2.857143,6.0
555,-1631.117894,5.4046,2.3603,0.7265,0.7787,22.0055,7.9161,2.0393,0.3318,0.4053,...,6.0,13.0,22.0,7.0,22.0,8.0,5.0,2.0,2.857143,6.0
556,-1631.116032,6.1234,1.7018,0.7203,0.7720,21.8155,7.8478,2.1136,0.2467,0.4688,...,6.0,13.0,18.0,15.0,19.0,7.0,5.0,2.0,2.857143,4.0
557,-1631.116753,5.8617,1.9124,0.7195,0.7711,21.7911,7.8390,2.0194,0.3286,0.4014,...,8.0,13.0,21.0,10.0,17.0,8.0,6.0,2.0,2.857143,6.0
